In [1]:
import pandas as pd

df=pd.read_excel("Web_Series.xlsx")

df.head()

,Series_Name,Platform,Genre,Country,Seasons,Episodes,Release_Year,IMDB_Rating,Viewership_Millions,Average_Episode_Length_Min
0,Breaking Bad,Netflix,Drama,USA,1,10,2005,7.5,10.0,35
1,Stranger Things,Amazon Prime,Thriller,UK,2,11,2006,7.6,11.2,36
2,Money Heist,Disney+ Hotstar,Crime,India,3,12,2007,7.7,12.4,37
3,Dark,HBO,Action,Spain,4,13,2008,7.8,13.6,38
4,The Witcher,Apple TV+,Sci-Fi,Germany,5,14,2009,7.9,14.8,39


In [2]:
#Number of rows and columns
rows,columns=df.shape

print("Number of rows:  ",rows)
print("Number of columns:  ",columns)

Number of rows:   100
Number of columns:   10


In [3]:
#Table of column names and data types
column_info=pd.DataFrame({
    "Column Name": df.columns,
    "Data Type":df.dtypes.values
})

column_info

,Column Name,Data Type
0,Series_Name,object
1,Platform,object
2,Genre,object
3,Country,object
4,Seasons,int64
5,Episodes,int64
6,Release_Year,int64
7,IMDB_Rating,float64
8,Viewership_Millions,float64
9,Average_Episode_Length_Min,int64


In [4]:
#Checking missing values
df.isnull().sum()

,0
Series_Name,0
Platform,0
Genre,0
Country,0
Seasons,0
Episodes,0
Release_Year,0
IMDB_Rating,0
Viewership_Millions,0
Average_Episode_Length_Min,0


In [5]:
#Columns having missing values
missing_columns=df.columns[df.isnull().any()]
missing_columns

Index([], dtype='object')

In [6]:
numerical_cols = df.select_dtypes(include=['int64','float64']).columns

#Filling missing values with mean
for col in numerical_cols:
  df[col].fillna(df[col].mean(),inplace=True)

/tmp/ipython-input-2453653236.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(),inplace=True)


In [7]:
#Checking for duplicate rows

df.duplicated().sum()

np.int64(0)

In [8]:
#Counting duplicate rows
duplicate_count=df.duplicated().sum()
print("Number od duplicate rows: ",duplicate_count)



Number od duplicate rows:  0


In [9]:
df=df.drop_duplicates()

print("Updated dataset shape(rows,columns):",df.shape)

Updated dataset shape(rows,columns): (100, 10)


In [10]:
#Converting numerical columns to numeric type

num_cols=[
    "Seasons","Episodes","Release_Year",
    "IMDB_Rating","Viewership_Millions",
    "Average_Episode_Length_Min"
]

for col in num_cols:
  df[col]=pd.to_numeric(df[col],errors="coerce")

In [11]:
#Converting Release_Year to Date Format
df["Release_Date"]=pd.to_datetime(df["Release_Year"],format="%Y")

In [12]:
df["Release_Year_Extracted"]=df["Release_Date"].dt.year
df["Release_Month"]=df["Release_Date"].dt.month
df["Release_Day"]=df["Release_Date"].dt.day

df.dtypes

,0
Series_Name,object
Platform,object
Genre,object
Country,object
Seasons,int64
Episodes,int64
Release_Year,int64
IMDB_Rating,float64
Viewership_Millions,float64
Average_Episode_Length_Min,int64


In [13]:
column="IMDB_Rating"

#Calculating Q1,Q3 and IQR
Q1=df[column].quantile(0.25)
Q3=df[column].quantile(0.75)
IQR=Q3-Q1

#Defining lower and upper bounds
lower_bound = Q1-1.5 * IQR
upper_bound = Q3+1.5 * IQR

#Identifying outliers
outliers=df[(df[column]<lower_bound) | (df[column]) > upper_bound]

print("Number of outliers:",outliers.shape[0])
outliers



Number of outliers: 0


,Series_Name,Platform,Genre,Country,Seasons,Episodes,Release_Year,IMDB_Rating,Viewership_Millions,Average_Episode_Length_Min,Release_Date,Release_Year_Extracted,Release_Month,Release_Day


In [14]:
#Apply capping
df[column]=df[column].clip(lower_bound,upper_bound)

new_outliers=df[(df[column]<lower_bound) | (df[column]>upper_bound)]
print("Outliers after Treatment:",new_outliers.shape[0])

Outliers after Treatment: 0


In [15]:
cols=["IMDB_Rating","Viewership_Millions"]

#Calculate desciptive Statistics
stats=df[cols].agg(
    ["mean","median","std","min","max"]
)

stats

,IMDB_Rating,Viewership_Millions
mean,8.560000,55.561000
median,8.500000,52.600000
std,0.700937,30.938831
min,7.500000,10.000000
max,9.900000,112.000000


In [19]:
#Group By Platform and apply aggregation functions

grouped_data=df.groupby("Platform").agg(
    Series_Count=("Series_Name","count"),
    Total_Viewership=("Viewership_Millions","sum"),
    Average_IMBD_Rating=("IMDB_Rating","mean"),
    Max_IMBD_Rating=("IMDB_Rating","max")
)
grouped_data

,Series_Count,Total_Viewership,Average_IMBD_Rating,Max_IMBD_Rating
Platform,,,,
Amazon Prime,20,1071.2,8.465000,9.6
Apple TV+,19,1111.9,8.752632,9.9
Disney+ Hotstar,20,1094.9,8.565000,9.7
HBO,20,1118.6,8.665000,9.8
Netflix,21,1159.5,8.371429,9.5
